In [ ]:
import numpy as np
import time
import random
import os
import matplotlib.pyplot as plt

import torch
import torch.optim as optim
from torchvision import utils
from torch.utils.data import Dataset, DataLoader

from dataloader import *
from models import *
from utils import *

random_seed = 777

rand_fix(random_seed)

device = torch.device("cuda:4" if (torch.cuda.is_available()) else "cpu")

dataset_dir = "./Datasets/"
dataset_dir_made = './dataset'
save_dir = "./CheckPoint4/"
num_workers = 0
batch_size = 8
num_epochs = 20

lr = 0.001


transform = trans(mode = 'normal')
transform_mask = trans(mode = 'mask')
transform_test = trans(mode = 'test')

train_dataset = FaceMask(dataset_dir, dataset_dir_made, transform, transform_mask)
train_dataloader = DataLoader(train_dataset, batch_size = batch_size,
                             shuffle = True, num_workers = num_workers)

test_dataset = FaceMask(dataset_dir, dataset_dir_made, transform, transform_test = transform_test, test = True)
test_dataloader = DataLoader(test_dataset, batch_size = 8,
                            shuffle = False, num_workers = num_workers)

sample = next(iter(test_dataloader))
test_img = sample['test_img'].to(device)
show_img = sample['show_img']

model = Unet().to(device)
model.apply(weights_init)

optimizer = optim.Adam(model.parameters(), lr = lr, betas = (0.5,0.999))

loss = nn.BCELoss()

Start = time.time()
print("Starting Training Loop...")
Loss_list = []

for epoch in range(num_epochs):
    print("Epoch {}/{}".format(epoch+1, num_epochs))
    print("-" * 10)
    start = time.time()
    
    face_masked = None
    face_cloth = None
    face_surgical = None
    
    mask = None
    mask_cloth = None
    mask_surgical = None
    for i, sample in enumerate(train_dataloader, 0):
        face_masked, mask = sample['W_mask'], sample['mask']
        face_cloth, face_surgical = sample['cloth'], sample['surgical']
        mask_cloth, mask_surgical = sample['mask_cloth'], sample['mask_surgical']
        batch_size = mask.size(0)
        
        face_masked = face_masked.to(device)
        mask = mask.to(device)
        face_cloth = face_cloth.to(device)
        face_surgical = face_surgical.to(device)
        mask_cloth = mask_cloth.to(device)
        mask_surgical = mask_surgical.to(device)
        
        optimizer.zero_grad()
        
        output_m = model(face_masked)
        output_c = model(face_cloth)
        output_s = model(face_surgical)
        Loss_m = loss(output_m, mask)
        Loss_c = loss(output_c, mask_cloth)
        Loss_s = loss(output_s, mask_surgical)
        
        Loss = Loss_m + Loss_c + Loss_s
        Loss_list.append(Loss.item())
        Loss.backward()
        optimizer.step()
        
        if (i % 100 == 0):
            print("[{:d}/{:d}] Loss : {:.4f}".format(i, len(train_dataloader), Loss.item()))
        

    save_checkpoint({
        'epoch' : epoch + 1,
        'model_state_dict' : model.state_dict(),
        'model_opt' : optimizer.state_dict()
    }, save_dir, epoch + 1)
    
    print("=" * 100)
    print('Time taken by epoch: {:.0f}h {:.0f}m {:.0f}s'.format(((time.time() - start) // 60) // 60, (time.time() - start) // 60, (time.time() - start) % 60))
    print()
    
    with torch.no_grad():
        result = model(face_masked).cpu()
        result_s = model(face_surgical).cpu()
        result_c = model(face_cloth).cpu()
        test_result = model(test_img).cpu()
        m = mask.cpu()
        s = mask_surgical.cpu()
        c = mask_cloth.cpu()
        sample = []
        test_sample = []
        for i in range(batch_size):
            sample.extend([m[i], result[i], s[i], result_s[i], c[i], result_c[i]])
        for i in range(8):
            test_sample.append(test_result[i])

        result_img = utils.make_grid(sample, padding = 2,
                                        normalize = True, nrow = 6)
        test_result_img = utils.make_grid(test_sample, padding = 0,
                                        normalize = True, nrow = 1)

        utils.save_image(result_img, "./result_unet/result-{}epoch.png".format(epoch + 1))
        utils.save_image(test_result_img, "./result_unet/test_result-{}epoch.png".format(epoch + 1))
        
print("Training is finished")
hour = ((time.time() - Start) // 60) // 60
print('Time taken by num_epochs: {:.0f}h {:.0f}m {:.0f}s'.format(hour, (time.time() - Start) - hour * 60, (time.time() - Start) % 60))

Starting Training Loop...
Epoch 1/20
----------
[0/623] Loss : 2.3319
[100/623] Loss : 0.5120
[200/623] Loss : 0.2134
[300/623] Loss : 0.1555
[400/623] Loss : 0.1590
[500/623] Loss : 0.2724
[600/623] Loss : 0.1344
--- checkpoint saved to ./CheckPoint4/checkpoint-1.pt ---
Time taken by epoch: 0h 15m 48s

Epoch 2/20
----------
[0/623] Loss : 0.1620
[100/623] Loss : 0.1190
[200/623] Loss : 0.1168
